In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import os

FILE_NAME = "marks.csv"

class StudentGradingBackend:
    def __init__(self):
        self.data = self.load_data()

    def calculate_grade(self, marks: float) -> str:
        if marks >= 90:
            return "A+"
        elif marks >= 80:
            return "A"
        elif marks >= 70:
            return "B+"
        elif marks >= 60:
            return "B"
        elif marks >= 50:
            return "C"
        else:
            return "F"

    def add_student(self, roll_no: str, name: str, marks: float):
        if not roll_no or not name:
            raise ValueError("Roll number and name cannot be empty.")
        if not (0 <= marks <= 100):
            raise ValueError("Marks must be between 0 and 100.")
        grade = self.calculate_grade(marks)
        new_record = {"Roll No": roll_no, "Name": name, "Marks": marks, "Grade": grade}
        self.data = pd.concat([self.data, pd.DataFrame([new_record])], ignore_index=True)
        return new_record

    def show_all_students(self):
        if self.data.empty:
            return "No student records found."
        return self.data.copy()

    def get_statistics(self):
        if self.data.empty:
            return {"average": 0, "topper": None}
        avg = self.data["Marks"].mean()
        topper_row = self.data.loc[self.data["Marks"].idxmax()]
        topper = {
            "Name": topper_row["Name"],
            "Roll No": topper_row["Roll No"],
            "Marks": topper_row["Marks"],
            "Grade": topper_row["Grade"],
        }
        return {"average": avg, "topper": topper}

    def save_data(self):
        self.data.to_csv(FILE_NAME, index=False)
        return f"Data saved successfully to {FILE_NAME}"

    def load_data(self):
        if os.path.exists(FILE_NAME):
            try:
                return pd.read_csv(FILE_NAME)
            except Exception:
                return pd.DataFrame(columns=["Roll No", "Name", "Marks", "Grade"])
        else:
            return pd.DataFrame(columns=["Roll No", "Name", "Marks", "Grade"])

    def clear_all_data(self):
        self.data = pd.DataFrame(columns=["Roll No", "Name", "Marks", "Grade"])
        if os.path.exists(FILE_NAME):
            os.remove(FILE_NAME)
        return "All records deleted successfully."

    def search_student(self, keyword: str):
        if self.data.empty:
            return "No records found."
        keyword = keyword.lower()
        results = self.data[
            self.data["Name"].str.lower().str.contains(keyword)
            | self.data["Roll No"].astype(str).str.lower().str.contains(keyword)
        ]
        if results.empty:
            return f"No student found for keyword '{keyword}'."
        return results


class StudentGradingGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Student Grading System")
        self.root.geometry("800x550")
        self.root.resizable(False, False)
        self.backend = StudentGradingBackend()

        tk.Label(root, text="STUDENT GRADING SYSTEM", font=("Arial", 18, "bold"), fg="#1A73E8").pack(pady=10)

        form = tk.Frame(root, bg="#f9f9f9", padx=10, pady=10)
        form.pack(pady=10)

        tk.Label(form, text="Roll No:", font=("Arial", 11)).grid(row=0, column=0, padx=8, pady=5)
        tk.Label(form, text="Name:", font=("Arial", 11)).grid(row=1, column=0, padx=8, pady=5)
        tk.Label(form, text="Marks:", font=("Arial", 11)).grid(row=2, column=0, padx=8, pady=5)

        self.roll_entry = tk.Entry(form, width=25)
        self.name_entry = tk.Entry(form, width=25)
        self.marks_entry = tk.Entry(form, width=25)

        self.roll_entry.grid(row=0, column=1)
        self.name_entry.grid(row=1, column=1)
        self.marks_entry.grid(row=2, column=1)

        btn_frame = tk.Frame(root)
        btn_frame.pack(pady=10)

        tk.Button(btn_frame, text="Add Student", command=self.add_student, bg="#4CAF50", fg="white", width=14).grid(row=0, column=0, padx=5)
        tk.Button(btn_frame, text="Show All", command=self.show_all, bg="#2196F3", fg="white", width=14).grid(row=0, column=1, padx=5)
        tk.Button(btn_frame, text="Statistics", command=self.show_stats, bg="#9C27B0", fg="white", width=14).grid(row=0, column=2, padx=5)
        tk.Button(btn_frame, text="Search", command=self.search_student, bg="#607D8B", fg="white", width=14).grid(row=0, column=3, padx=5)
        tk.Button(btn_frame, text="Save Data", command=self.save_data, bg="#FF9800", fg="white", width=14).grid(row=0, column=4, padx=5)

        self.tree = ttk.Treeview(root, columns=("Roll No", "Name", "Marks", "Grade"), show="headings", height=12)
        self.tree.heading("Roll No", text="Roll No")
        self.tree.heading("Name", text="Name")
        self.tree.heading("Marks", text="Marks")
        self.tree.heading("Grade", text="Grade")
        for col in ("Roll No", "Name", "Marks", "Grade"):
            self.tree.column(col, width=150, anchor="center")
        self.tree.pack(pady=10)

        self.status_label = tk.Label(root, text="Welcome! Add student data to begin.", fg="#555", font=("Arial", 10))
        self.status_label.pack(side="bottom", pady=10)

        self.show_all()

    def add_student(self):
        roll = self.roll_entry.get().strip()
        name = self.name_entry.get().strip()
        marks = self.marks_entry.get().strip()
        if not roll or not name or not marks:
            messagebox.showwarning("Input Error", "Please fill all fields.")
            return
        try:
            marks = float(marks)
            new_record = self.backend.add_student(roll, name, marks)
            messagebox.showinfo("Success", f"Added {new_record['Name']} with Grade {new_record['Grade']}")
            self.status_label.config(text=f"Added {name} with grade {new_record['Grade']}")
            self.clear_entries()
            self.show_all()
        except ValueError as e:
            messagebox.showerror("Invalid Input", str(e))

    def show_all(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        df = self.backend.show_all_students()
        if isinstance(df, str):
            self.status_label.config(text=df)
            return
        for _, row in df.iterrows():
            self.tree.insert("", "end", values=list(row))
        self.status_label.config(text=f"Displaying {len(df)} records.")

    def show_stats(self):
        stats = self.backend.get_statistics()
        if stats["average"] == 0:
            messagebox.showinfo("No Data", "No student records to analyze.")
            return
        topper = stats["topper"]
        msg = f"Average Marks: {stats['average']:.2f}\\nTopper: {topper['Name']} ({topper['Marks']} marks, Grade {topper['Grade']})"
        messagebox.showinfo("Class Statistics", msg)

    def search_student(self):
        keyword = simpledialog.askstring("Search Student", "Enter name or roll no to search:")
        if not keyword:
            return
        result = self.backend.search_student(keyword)
        if isinstance(result, str):
            messagebox.showinfo("Search Result", result)
            return
        for i in self.tree.get_children():
            self.tree.delete(i)
        for _, row in result.iterrows():
            self.tree.insert("", "end", values=list(row))
        self.status_label.config(text=f"Showing {len(result)} result(s) for '{keyword}'.")

    def save_data(self):
        msg = self.backend.save_data()
        messagebox.showinfo("Saved", msg)
        self.status_label.config(text=msg)

    def clear_entries(self):
        self.roll_entry.delete(0, tk.END)
        self.name_entry.delete(0, tk.END)
        self.marks_entry.delete(0, tk.END)


root = tk.Tk()
app = StudentGradingGUI(root)
root.mainloop()
